In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

##### [1. Forward Imaging Model](fwdimaging.ipynb)

## 2. The Recovery Process

This process recovers the Hi-Res complex object using the Lo-Res image set. Each image of it is generated by the lighting that emerges from a different LED of a LED illuminator during the forward imaging process ([fwdimaging.ipynb](fwdimaging.ipynb)).

In [ ]:
import fwdimaging as fwd
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import os
import math
from PIL import Image

In [ ]:
%matplotlib inline

In [ ]:
def select_centered_subset(n_leds, leds2use, **kwargs):
    """Returns a list of integers that represents a centered subset of
    indexes from a set of images' indexes.
    
    ONLY USE IT when you simulate a LED illuminator type 'GRID'.
    
    n_leds ('int') is the lenght of n-by-n array. leds2use ('int')
    is the approximate length of the subset. It will be recalculated
    if is necessary (line 28)."""
    firstis1 = kwargs.pop('firstis1', False)
    # Begin the selection down ('int') rows down.
    down = kwargs.pop('down', 0)
    # Begin the selection right ('int') columns right.
    right = kwargs.pop('right', 0)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(select_centered_subset.__name__, list(kwargs.keys())[-1]))
    # Number of LEDs per row or per column in a squared LED grid.
    leds_prc = int(math.sqrt(n_leds))
    new_leds_prc = int(math.sqrt(leds2use))
    if firstis1:
        arr = np.arange(1, n_leds + 1) 
    else:
        arr = np.arange(n_leds)
    # Convert arr ('numpy.ndarray') (one-dimensional) in 
    # leds_prc-by-leds_prc array (two-dimensional).
    arr = np.split(arr, leds_prc)  # Here, arr is type 'list'.
    arr = np.array(arr)  # arr is a numpy.ndarray.
    if (leds_prc % 2 == 0 and new_leds_prc % 2 != 0) or (leds_prc % 2 != 0 and new_leds_prc % 2 == 0):
        new_leds_prc -= 1  # leds_prc ('int') and new_leds_prc ('int')
                           # are even or odd.
    # leds_prc ('int') - new_leds_prc ('int') is always even.
    limit_l_up = (leds_prc - new_leds_prc) // 2
    limit_r_down = leds_prc - limit_l_up
    arr = arr[limit_l_up - down:limit_r_down - down, limit_l_up + right:limit_r_down + right]
    arr = arr.flat
    return list(arr)

In [ ]:
def get_set_from_folder(subsel, **kwargs):
    """Returns an array with the Lo-Res images as arrays and a list
    with the indexes of np.zeros() in the l_imgs ('list')."""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'lores-set-0001')
    # Path of the directory that contains the low-resolution image
    # set.
    path = kwargs.pop('path', os.path.join(s3out,
                                           'microscope',
                                           dirname))
    prefix = kwargs.pop('prefix', 'lores-img_')
    extension = kwargs.pop('extension', '.tiff')
    lensuffix = kwargs.pop('lensuffix', len(str(max(subsel))))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_set_from_folder.__name__, list(kwargs.keys())[-1]))
    l_imgs = list()
    l_idx_zeros = list()
    for i in range(len(subsel)):
        filepath = os.path.join(path,
                                prefix + fwd.num_str_zeros(subsel[i],
                                    lensuffix) + extension)
        try:
            img = Image.open(filepath)
            arr_img = np.array(img)
            print('Opened Lo-Res image:', filepath)
            img.close()
            l_imgs.append(arr_img)
        except IOError:
            print('No such file:', filepath)
            l_imgs.append(np.zeros((0, 0)))
            l_idx_zeros.append(i)
    return np.array(l_imgs), l_idx_zeros

In [ ]:
def get_sequence(l_idx, **kwargs):
    """Returns the reconstruction sequence as list of integer numbers.
    Each int represents the index of a picture in a ordered list of
    images.
    
    The way is spiral-shaped.
    
    For squared arrays with odd number of rows, the spiral's head is
    in the center; instead, for an even number, the head is in the
    (center, center - 1) coordinate ((column, row))."""
    clockwise = kwargs.pop('clockwise', True)
    # n2max = kwargs.pop('n2max', leds_prc ** 2 - 1)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_sequence.__name__, list(kwargs.keys())[-1]))
    n = int(math.ceil(math.sqrt(len(l_idx))))
    # Complete with ones l_idx ('list') to convert it into squared
    # array.
    arr = np.ones(n ** 2, dtype=int)
    arr *= -1
    for i in range(len(l_idx)):
        arr[i] = l_idx[i]
    # Split arr ('numpy.ndarray') to give it n-by-n new order.
    arr = np.split(arr, n)
    arr = np.array(arr)
    # Determinate the (x, y) coordinate ((colum, row)) of the head the
    # spiral.
    if n % 2 != 0:
        col = (n - 1) // 2
        row = col
    else:
        col = (n // 2) - 1
        row = col
    addends = [(-1, -1), (1, 1)]
    if clockwise:
        i = 0  # Index in addends ('list').
    else:
        i = 1
    c = 1
    l_seq = list()
    l_seq.append(arr[row][col])  # First, add the head of the spiral to
                             # the list.
    j = 1
    steps = 1
    while True:
        i %= 2
        for k in range(j):
            col += addends[i][0]
            el = arr[row][col]
            if el != -1:  # It does not add the -1 index.
                l_seq.append(el)
            steps += 1
            if steps >= arr.size:
                break
        if steps >= arr.size:
            break
        for k in range(j):
            row += addends[i][1]
            el = arr[row][col]
            if el != -1:
                l_seq.append(arr[row][col])
            steps += 1
            if steps >= arr.size:
                break
        if steps >= arr.size:
            break
        i += 1
        j += 1
    return l_seq

In [ ]:
def get_photo(**kwargs):
    """Returns the Hi-Res object recovered by applying of Fourier
    Ptychography algorithm."""
    inpath = kwargs.pop('inpath',
                        os.path.join(os.path.expanduser('~'),
                                     's3-out',
                                     'microscope',
                                     'lores-set-0001'))
    outdirname = kwargs.pop('outdirname', os.path.join(inpath,
                                                       'hq-fp-0001'))
    outpath = kwargs.pop('outpath', fwd.find_out_dir(dirname=outdirname,
                                    parentdir='microscope'))
    inprefix = kwargs.pop('prefix', 'lores-img_')
    ext = kwargs.pop('ext', '.tif')  # Extension of the input files.
    firstsuffix = kwargs.pop('firstsuffix', '001')
    illuminator = kwargs.pop('illuminator', 'GRID')
    # For a LED illuminator type 'GRID', leds ('int') is the number of
    # LEDs per row or per column of a squared LED grid, e.g. 15;
    # instead, for a one type 'RING', leds is a list that contains the
    # number of LEDs by ring from the center to the edge of the LED
    # illuminator, e.g., [12, 18, 36].
    leds = kwargs.pop('leds', 15)
    # leds2use ('int') is the ORDER OF A SQUARED SUBSET of Lo-Res image
    # set. This subset is aligned to the center of the main set.
    # leds2use must be less than or equal to n ('int').
    leds2use = kwargs.pop('leds2use', -1)
    d = kwargs.pop('d', 4)  # Distance in mm between LEDs.
    # Distance in mm between the LED grid and the sample.
    h = kwargs.pop('h', 90)
    # Does the first image have 1 at the end of its filename?
    namesfrom1 = kwargs.pop('namesfrom1', True)
    ##
    # Determination of default valor for subsel ('list').
    l_subsel_idxs = list()
    if isinstance(leds, int):  # Default subset for illuminator type
                               # 'GRID'.
        if leds2use == -1 or leds2use > leds:
            leds2use = leds
        l_subsel_idxs = select_centered_subset(leds ** 2,
                                               leds2use ** 2,
                                               firstis1=namesfrom1)
    elif isinstance(leds, list):  # Default subset for illuminator
                                  # type 'RING'.
        if leds2use in leds:
            if namesfrom1:
                l_subsel_idxs = list(range(1, leds2use + 1))
            else:
                l_subsel_idxs = list(range(leds2use))
        elif namesfrom1:
            l_subsel_idxs = list(range(1, sum(leds) + 1))
        else:
            l_subsel_idxs = list(range(sum(leds)))
    subsel = kwargs.pop('subsel', l_subsel_idxs)
    ##
    # Parameters of the coherent imaging system.
    wavelen = kwargs.pop('wavelen', 0.63e-6)
    # Sampling pixel size of the CCD.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    # DANGER! IT COULD PIXELATE the output image.
    zoom = kwargs.pop('zoom', 4)
    loops = kwargs.pop('loops', 5)
    # Show the output Hi-Res image in the notebook.
    hqname = kwargs.pop('hqname', 'hq-img')  # Name of the file.
    # Does it the output image in the notebook of the calling?
    show = kwargs.pop('show', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_photo.__name__, list(kwargs.keys())[-1]))
    l_xy_leds = list()
    if illuminator.lower() == 'grid' and isinstance(leds, int):
        l_xy_leds = fwd.gen_xy_led_grid(leds2use, d)
    elif illuminator.lower() == 'ring' and isinstance(leds, list):
        l_xy_leds = fwd.gen_xy_led_ring(leds, d)
    wvs = fwd.generate_wave_vectors(l_xy_leds, h)
    lores_imgs_seq, idx_zeros = get_set_from_folder(subsel,
                                    path=inpath,
                                    prefix=inprefix,
                                    extension=ext,
                                    lensuffix=len(firstsuffix))
    # Search for the height and the width in pixels of a Lo-Res
    # picture.
    hpx = 0
    wpx = 0
    for i in range(len(lores_imgs_seq)):
        if i not in idx_zeros:
            hpx, wpx = lores_imgs_seq[i].shape
            break
    # np.zeros((0, 0)) arrays in lores_imgs_seq ('numpy.ndarray') are
    # replaced by other array from the same kind and wpx-by-hpx shape.
    for i in idx_zeros:
        lores_imgs_seq[i] = np.zeros((hpx, wpx))
    hpx *= zoom  # Height of the output Hi-Res image.
    wpx *= zoom  # ... the width.
    p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx,
                                              zoom, na, wpx, hpx)
    seq = get_sequence(subsel)
    obj_recover = np.ones((hpx, wpx), dtype=np.int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in range(len(seq)):
            k = seq[j]
            k -= 1
            kxc = fwd.round_half_up((wpx + 1) / 2.0 + kx[k] / dkx)
            kyc = fwd.round_half_up((hpx + 1) / 2.0 + ky[k] / dky)
            kyl = fwd.round_half_up(kyc - (p - 1) / 2.0)
            kyh = fwd.round_half_up(kyc + (p - 1) / 2.0)
            kxl = fwd.round_half_up(kxc - (q - 1) / 2.0)
            kxh = fwd.round_half_up(kxc + (q - 1) / 2.0)
            lores_ft = ((p / hpx) ** 2) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh]
            lores_ft *= cft
            img_lores = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            img_lores = ((hpx / p) ** 2) * lores_imgs_seq[k] * np.exp(1j * np.angle(img_lores))
            lores_ft = np.fft.fftshift(np.fft.fft2(img_lores)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = (1 - cft) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft
    filename = hqname + ext
    img_path = os.path.join(outpath, filename)
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    scipy.misc.toimage(obj_recover, cmin=0, cmax=255).save(img_path)
    print('Saved high-quality image:', img_path)
    if show:
        plt_img = plt.imshow(obj_recover, cmap='Greys_r')
    return obj_recover